In [45]:
# Import dependencies
import pandas as pd
import numpy as np
import tensorflow as tf

In [46]:
# Import cleaned data
data = pd.read_csv("./cleanData.csv")

In [47]:
# Split data into training and validation
n = len(data)
train_data = data[0:int(n*0.8)]
val_data = data[int(n*0.8):]

num_features = data.shape[1]

In [48]:
# Normalize data
train_mean = train_data.mean()
train_std = train_data.std()
train_mean.to_json('LSTMjs/mean.json')
train_std.to_json('LSTMjs/std.json')

train_data = (train_data - train_mean) / train_std
val_data = (val_data - train_mean) / train_std

In [49]:
# Window class for data training
class WindowGenerator():
  def __init__(self, input_width, label_width, shift,
               train_data=train_data, val_data=val_data,
               label_columns=None):
    # Store the raw data.
    self.train_data = train_data
    self.val_data = val_data

    # Work out the label column indices.
    self.label_columns = label_columns
    if label_columns is not None:
      self.label_columns_indices = {name: i for i, name in
                                    enumerate(label_columns)}
    self.column_indices = {name: i for i, name in
                           enumerate(train_data.columns)}

    # Work out the window parameters.
    self.input_width = input_width
    self.label_width = label_width
    self.shift = shift

    self.total_window_size = input_width + shift

    self.input_slice = slice(0, input_width)
    self.input_indices = np.arange(self.total_window_size)[self.input_slice]

    self.label_start = self.total_window_size - self.label_width
    self.labels_slice = slice(self.label_start, None)
    self.label_indices = np.arange(self.total_window_size)[self.labels_slice]

In [50]:
# Window class function for partitioning
def split_window(self, features):
  inputs = features[:, self.input_slice, :]
  labels = features[:, self.labels_slice, :]
  if self.label_columns is not None:
    labels = tf.stack(
        [labels[:, :, self.column_indices[name]] for name in self.label_columns],
        axis=-1)

  inputs.set_shape([None, self.input_width, None])
  labels.set_shape([None, self.label_width, None])

  return inputs, labels

WindowGenerator.split_window = split_window

In [51]:
# Window class function for making datasets
def make_dataset(self, data):
  data = np.array(data, dtype=np.float32)
  ds = tf.keras.utils.timeseries_dataset_from_array(
      data=data,
      targets=None,
      sequence_length=self.total_window_size,
      sequence_stride=1,
      shuffle=True,
      batch_size=32,)

  ds = ds.map(self.split_window)

  return ds

WindowGenerator.make_dataset = make_dataset

In [52]:
# Window class functions for accessing specific datasets
@property
def train(self):
  return self.make_dataset(self.train_data)

@property
def val(self):
  return self.make_dataset(self.val_data)

WindowGenerator.train = train
WindowGenerator.val = val

In [53]:
# Creation of our specific window
wide_window = WindowGenerator(
    input_width=24, label_width=24, shift=1,
    label_columns=['Wx','Wy','Temp'])

In [54]:
# Defining a training function for the model
MAX_EPOCHS = 20

def compile_and_fit(model, window, patience=2):
  early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                    patience=patience,
                                                    mode='min')

  model.compile(loss=tf.keras.losses.MeanSquaredError(),
                optimizer=tf.keras.optimizers.Adam(),
                metrics=[tf.keras.metrics.MeanAbsoluteError()])

  model.fit(window.train, epochs=MAX_EPOCHS,
                      validation_data=window.val,
                      callbacks=[early_stopping])

In [55]:
# Model layer specification
lstm_model = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(32, return_sequences=True),
    tf.keras.layers.Dense(units=3)
])

In [56]:
# Train the model
compile_and_fit(lstm_model, wide_window)

Epoch 1/20
6794/6794 [==============================] - 182s 26ms/step - loss: 0.2545 - mean_absolute_error: 0.3018 - val_loss: 0.2534 - val_mean_absolute_error: 0.3034
Epoch 2/20
6794/6794 [==============================] - 159s 23ms/step - loss: 0.2357 - mean_absolute_error: 0.2873 - val_loss: 0.2510 - val_mean_absolute_error: 0.3013
Epoch 3/20
6794/6794 [==============================] - 167s 25ms/step - loss: 0.2331 - mean_absolute_error: 0.2856 - val_loss: 0.2496 - val_mean_absolute_error: 0.3012
Epoch 4/20
6794/6794 [==============================] - 174s 26ms/step - loss: 0.2316 - mean_absolute_error: 0.2847 - val_loss: 0.2498 - val_mean_absolute_error: 0.3017
Epoch 5/20
6794/6794 [==============================] - 177s 26ms/step - loss: 0.2306 - mean_absolute_error: 0.2841 - val_loss: 0.2493 - val_mean_absolute_error: 0.3011
Epoch 6/20
6794/6794 [==============================] - 187s 28ms/step - loss: 0.2298 - mean_absolute_error: 0.2836 - val_loss: 0.2494 - val_mean_absolute_

In [57]:
# Save the model for import to TensorflowJS
import tensorflowjs as tfjs

tfjs.converters.save_keras_model(lstm_model, "./LSTMjs")

NotFoundError: c:\Users\Lande\AppData\Local\Programs\Python\Python311\Lib\site-packages\tensorflow_decision_forests\tensorflow\ops\inference\inference.so not found